In [23]:
import pandas as pd
import numpy as np
import math
import pickle
import torch
import random
import torch.nn as nn
import torch.optim as optim
import time
from torch.utils.data import Dataset, DataLoader
from models.BertSeqTransformer import StandardTransformer
from datasets.SpotifyDataset import SpotifyDataset, bert_collate_fn, custom_collate_fn
N=100000
torch.manual_seed(1)
EPOCHS = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)

You are using device: cpu


In [17]:
print("READING THE DATA")
with open("data/all_session_tracks_train.pkl", 'rb') as f:
    train_tracks = pickle.load(f)
    train_tracks = train_tracks[0:N]

with open("data/all_session_skips_train.pkl", 'rb') as f:
    train_skips = pickle.load(f)
    train_skips = train_skips[0:N]

with open("data/all_session_tracks_test.pkl", 'rb') as f:
    test_tracks = pickle.load(f)
    test_tracks = test_tracks[0:N]

with open("data/all_session_skips_test.pkl", 'rb') as f:
    test_skips = pickle.load(f)
    test_skips = test_skips[0:N]

with open("data/track_vocabs.pkl", 'rb') as f:
    track_vocab = pickle.load(f)

READING THE DATA


In [25]:
INPUT_SIZE = len(track_vocab)
OUTPUT_SIZE = len(track_vocab)
PAD_IDX = track_vocab['pad']
BATCH_SIZE = 256
MAX_LEN = 20
SKIP = False
PAD_MASK = 1

train_dataset = SpotifyDataset(train_tracks, train_skips, track_vocab, bert=False, bert_mask_proportion = 0.2, skip_pred=SKIP, padding=False)
valid_dataset = SpotifyDataset(test_tracks, test_skips, track_vocab, bert=False, bert_mask_proportion = 0.2, skip_pred=SKIP, padding=False)

train_loader = DataLoader(dataset = train_dataset, batch_size = BATCH_SIZE, shuffle = True, collate_fn =custom_collate_fn)
valid_loader = DataLoader(dataset = valid_dataset, batch_size = BATCH_SIZE, shuffle = False, collate_fn =custom_collate_fn)

#OPTIM PARAMETERS
learning_rate = 1e-5
transformer_seq_model = torch.load('output/model_bert_transformer_seq_embed_1e-5_256_dim.pth', map_location = ('cpu'))
transformer_seq_model.to(device)

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, transformer_seq_model.parameters()), lr=learning_rate)

if SKIP:
    criterion = nn.CrossEntropyLoss()
else:
    criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

train_losses, val_losses, train_accs, val_accs = [], [], [], []

1
BERT MASK PROB
0.2
1
BERT MASK PROB
0.2


In [33]:
with torch.no_grad():
    for batch_idx, (input_sequence, input_skips, label_sequence, label_skips) in enumerate(valid_loader):
        input_sequence = input_sequence.to(device)
        if SKIP==False:
                label = label_sequence.cuda()
                output_tokens, outputs = transformer_seq_model.greedy_decoder(transformer_seq_model, input_sequence, 10, input_sequence[:,-1])

AssertionError: Torch not compiled with CUDA enabled

In [51]:
y = torch.randn(2,3)

In [52]:
x = []

In [53]:
x.append(y)
x.append(y)

In [54]:
torch.Tensor(x)

ValueError: only one element tensors can be converted to Python scalars

In [56]:
z = torch.cat(x, dim=0)

In [59]:
z

tensor([[ 0.3255, -0.4791,  1.3790],
        [ 2.5286,  0.4107, -0.9880],
        [ 0.3255, -0.4791,  1.3790],
        [ 2.5286,  0.4107, -0.9880]])